In [77]:
# 6.0001 Problem Set 3
#
# The 6.0001 Word Game
# Created by: Kevin Luu <luuk> and Jenna Wiens <jwiens>
#
# Name          : <your name>
# Collaborators : <your collaborators>
# Time spent    : <total time>

import math
import random
import string

VOWELS = 'aeiou'
CONSONANTS = 'bcdfghjklmnpqrstvwxyz'
HAND_SIZE = 7
all_letters = list(VOWELS+CONSONANTS)

SCRABBLE_LETTER_VALUES = {
    'a': 1, 'b': 3, 'c': 3, 'd': 2, 'e': 1, 'f': 4, 'g': 2, 'h': 4, 'i': 1, 'j': 8, 'k': 5, 'l': 1, 'm': 3, 'n': 1, 'o': 1, 'p': 3, 'q': 10, 'r': 1, 's': 1, 't': 1, 'u': 1, 'v': 4, 'w': 4, 'x': 8, 'y': 4, 'z': 10, '*':0
}

# -----------------------------------
# Helper code
# (you don't need to understand this helper code)

WORDLIST_FILENAME = "words.txt"

def load_words():
    """
    Returns a list of valid words. Words are strings of lowercase letters.
    
    Depending on the size of the word list, this function may
    take a while to finish.
    """
    
    print("Loading word list from file...")
    # inFile: file
    inFile = open(WORDLIST_FILENAME, 'r')
    # wordlist: list of strings
    wordlist = []
    for line in inFile:
        wordlist.append(line.strip().lower())
    print("  ", len(wordlist), "words loaded.")
    return wordlist

def get_frequency_dict(sequence):
    """
    Returns a dictionary where the keys are elements of the sequence
    and the values are integer counts, for the number of times that
    an element is repeated in the sequence.

    sequence: string or list
    return: dictionary
    """
    
    # freqs: dictionary (element_type -> int)
    freq = {}
    for x in sequence:
        freq[x] = freq.get(x,0) + 1
    return freq
	

# (end of helper code)
# -----------------------------------

#
# Problem #1: Scoring a word
#
def get_word_score(word, n):
    word.lower()
    letter_scores = sum([SCRABBLE_LETTER_VALUES[letter] for letter in word])
    points = letter_scores*(7*len(word)-3*(n-len(word)))
    return max(points, 1)

#
# Make sure you understand how this function works and what it does!
#
def display_hand(hand):
    """
    Displays the letters currently in the hand.

    For example:
       display_hand({'a':1, 'x':2, 'l':3, 'e':1})
    Should print out something like:
       a x x l l l e
    The order of the letters is unimportant.

    hand: dictionary (string -> int)
    """
    
    for letter in hand.keys():
        for j in range(hand[letter]):
             print(letter, end=' ')      # print all on the same line
    print()                              # print an empty line

#
# Make sure you understand how this function works and what it does!
# You will need to modify this for Problem #4.
#
def deal_hand(n):
    """
    Returns a random hand containing n lowercase letters.
    ceil(n/3) letters in the hand should be VOWELS (note,
    ceil(n/3) means the smallest integer not less than n/3).

    Hands are represented as dictionaries. The keys are
    letters and the values are the number of times the
    particular letter is repeated in that hand.

    n: int >= 0
    returns: dictionary (string -> int)
    """
    
    hand={}
    num_vowels = int(math.ceil(n / 3))

    for i in range(num_vowels-1):
        x = random.choice(VOWELS)
        hand[x] = hand.get(x, 0) + 1
    hand['*'] = 1
    
    for i in range(num_vowels, n):    
        x = random.choice(CONSONANTS)
        hand[x] = hand.get(x, 0) + 1
    
    return hand

#
# Problem #2: Update a hand by removing letters
#

from collections import Counter

def update_hand(hand, word):
    word_dic = Counter(word)
    new_hand = hand
    for letter in word_dic:
        if letter in hand:
            new_hand[letter] = max(hand[letter]-word_dic[letter], 0)
    return dict(new_hand)

#
# Problem #3: Test word validity
#
"""
def is_valid_word(word, hand, word_list):
    valid = True
    if word not in word_list:
        return False
    
    word_dic = Counter(word)
    for letter in word:
        if word_dic[letter] > hand.get(letter, 0):
            return False
    
    return valid
""""

def is_valid_word(word, hand, word_list):
    subs = [word.replace('*', vowel) for vowel in vowels]
    if len(set(subs).intersection(set(word_list))) == 0:
        return False
    
    word_dic = Counter(word)
    c = 0
    for letter in word:
        if letter in CONSONANTS:
            if word_dic[letter] > hand.get(letter, 0):
                return False
        else:
            if word_dic[letter] - 1 > hand.get(letter, 0):
                c+=1
    if c > 1:
        return False
    
    return True

#
# Problem #5: Playing a hand
#
def calculate_handlen(hand):
    return sum([hand[letter] for letter in hand])

def play_hand(hand, word_list):
    game = True
    total = 0
    n = calculate_handlen(hand)
    while game:
        display_hand(hand)
        print('Enter word, or \'!!\' to indicate that you are finished.')
        word = input()
        if word == '!!':
            print('Game over. Total: '+ str(total))
            game = False
            break
        isvalid = is_valid_word(word, hand, word_list)
        while isvalid == False:
            print('That is not a valid word. Please choose another word.')
            word = input()
            valid = is_valid_word(word, hand, word_list)
        total += get_word_score(word, len(word))
        print (word+ ' earned ' + str(get_word_score(word, len(word)))+ ' points. Total: ' + str(total))
        update_hand(hand, word)
        if set(hand.values()) == {0}:
            print('Ran out of letters. Total score: '+ str(total))
            game = False
    return total     
        
        

    """
    Allows the user to play the given hand, as follows:

    * The hand is displayed.
    
    * The user may input a word.

    * When any word is entered (valid or invalid), it uses up letters
      from the hand.

    * An invalid word is rejected, and a message is displayed asking
      the user to choose another word.

    * After every valid word: the score for that word is displayed,
      the remaining letters in the hand are displayed, and the user
      is asked to input another word.

    * The sum of the word scores is displayed when the hand finishes.

    * The hand finishes when there are no more unused letters.
      The user can also finish playing the hand by inputing two 
      exclamation points (the string '!!') instead of a word.

      hand: dictionary (string -> int)
      word_list: list of lowercase strings
      returns: the total score for the hand
      
    """
    
    # BEGIN PSEUDOCODE <-- Remove this comment when you implement this function
    # Keep track of the total score
    
    # As long as there are still letters left in the hand:
    
        # Display the hand
        
        # Ask user for input
        
        # If the input is two exclamation points:
        
            # End the game (break out of the loop)

            
        # Otherwise (the input is not two exclamation points):

            # If the word is valid:

                # Tell the user how many points the word earned,
                # and the updated total score

            # Otherwise (the word is not valid):
                # Reject invalid word (print a message)
                
            # update the user's hand by removing the letters of their inputted word
            

    # Game is over (user entered '!!' or ran out of letters),
    # so tell user the total score

    # Return the total score as result of function



#
# Problem #6: Playing a game
# 


#
# procedure you will use to substitute a letter in a hand
#

def substitute_hand(hand, letter):
    if letter in hand and hand[letter]!=0:
        new_letter = random.choice(all_letters)
        hand[new_letter] = hand[letter]
        hand.pop(letter)
    return hand
    """ 
    Allow the user to replace all copies of one letter in the hand (chosen by user)
    with a new letter chosen from the VOWELS and CONSONANTS at random. The new letter
    should be different from user's choice, and should not be any of the letters
    already in the hand.

    If user provide a letter not in the hand, the hand should be the same.

    Has no side effects: does not mutate hand.

    For example:
        substitute_hand({'h':1, 'e':1, 'l':2, 'o':1}, 'l')
    might return:
        {'h':1, 'e':1, 'o':1, 'x':2} -> if the new letter is 'x'
    The new letter should not be 'h', 'e', 'l', or 'o' since those letters were
    already in the hand.
    
    hand: dictionary (string -> int)
    letter: string
    returns: dictionary (string -> int)
    """
    
       
    
def play_game(word_list):
    game_total = 0
    player_subs = False
    print('How many hands would you like to play?')
    m = int(input())
    replayed = False
    for _ in range(m):
        hand = deal_hand(HAND_SIZE)
        display_hand(hand)
        if not player_subs:
            print('Would you like to subsitute a letter?')
            if input().lower() == 'yes':
                print('Which letter do you want to substitute out?')
                sub = input()
                hand = substitute(hand, sub)
                player_subs = True
                condition = True
        score = play_hand(hand, word_list)
        prev_game = game_total
        game_total += score
        new_total = 0
        if not replayed:
            print('Would you like to replay that hand?')
            if input().lower()=='yes':
                new_total = prev_game + play_hand(hand, word_list)
                game_total = max(new_total, game_total)
                replayed = True
        print('Final score: '+str(game_total))
        
    """
    Allow the user to play a series of hands

    * Asks the user to input a total number of hands

    * Accumulates the score for each hand into a total score for the 
      entire series
 
    * For each hand, before playing, ask the user if they want to substitute
      one letter for another. If the user inputs 'yes', prompt them for their
      desired letter. This can only be done once during the game. Once the
      substitue option is used, the user should not be asked if they want to
      substitute letters in the future.

    * For each hand, ask the user if they would like to replay the hand.
      If the user inputs 'yes', they will replay the hand and keep 
      the better of the two scores for that hand.  This can only be done once 
      during the game. Once the replay option is used, the user should not
      be asked if they want to replay future hands. Replaying the hand does
      not count as one of the total number of hands the user initially
      wanted to play.

            * Note: if you replay a hand, you do not get the option to substitute
                    a letter - you must play whatever hand you just had.
      
    * Returns the total score for the series of hands

    word_list: list of lowercase strings
    """
    


#
# Build data structures used for entire session and play game
# Do not remove the "if __name__ == '__main__':" line - this code is executed
# when the program is run directly, instead of through an import statement
#
if __name__ == '__main__':
    word_list = load_words()
    play_game(word_list)

SyntaxError: EOL while scanning string literal (Temp/ipykernel_11180/2563391311.py, line 157)

In [ ]:
all_letters = list(VOWELS+CONSONANTS)

def substitute_hand(hand, letter):
    if letter in hand and hand[letter]!=0:
        new_letter = random.choice(all_letters)
        hand[new_letter] = hand[letter]
        hand.pop(letter)
    return hand

#everything works fine below EXCEPT:
#if a player chooses to replay a hand, the hand they start with isn't a full hand
#rather, its the hand they ended with in their previous round
def play_game(word_list):
    game_total = 0
    player_subs = False
    print('How many hands would you like to play?')
    m = int(input())
    replayed = False
    for _ in range(m):
        hand = deal_hand(HAND_SIZE)
        original_hand = hand
        display_hand(hand)
        if not player_subs:
            print('Would you like to subsitute a letter?')
            if input().lower() == 'yes':
                print('Which letter do you want to substitute out?')
                sub = input()
                hand = substitute_hand(hand, sub)
                original_hand = hand
                player_subs = True
        score = play_hand(hand, word_list)
        prev_game = game_total
        game_total += score
        new_total = 0
        if not replayed:
            print('Would you like to replay that hand?')
            if input().lower()=='yes':               #debugging
                new_total = prev_game + play_hand(original_hand, word_list)
                game_total = max(new_total, game_total)
                replayed = True
        print('Final score: '+str(game_total))

In [98]:
play_game(word_list)

How many hands would you like to play?
2
o e a k t f l 
Would you like to subsitute a letter?
no
o e a k t f l 
Enter word, or '!!' to indicate that you are finished.
tea
tea earned 63 points. Total: 63
o k f l 
Enter word, or '!!' to indicate that you are finished.
!!
Game over. Total: 63
Would you like to replay that hand?
no
Final score: 63
e e i t b z p 
Would you like to subsitute a letter?
pee
e e i t b z p 
Enter word, or '!!' to indicate that you are finished.
pee
pee earned 105 points. Total: 105
i t b z 
Enter word, or '!!' to indicate that you are finished.
bit
bit earned 105 points. Total: 210
z 
Enter word, or '!!' to indicate that you are finished.
!!
Game over. Total: 210
Would you like to replay that hand?
j
Final score: 273


In [40]:
subs

['CCaD', 'CCeD', 'CCiD', 'CCoD', 'CCuD']

In [31]:
Counter('word')['w']

1